In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('train.csv')
X = df.drop('fake', axis=1)  # axis=1 代表欄
y = df['fake']
train_data, test_data, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [16]:
class Node:
    def __init__(self, attribute=None, threshold=None, value=None, left=None, right=None):
        """
        初始化節點
        :param attributete: 分割特徵的索引
        :param threshold: 分割的閾值
        :param value: 叶子节点的值（类别）
        :param left: 左子树
        :param right: 右子树
        """
        self.attribute = attribute
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right


class DecisionTree:

    def __init__(self, max_depth=None, min_samples_split=None):
        """
        初始化決策樹
        :param max_depth: 樹的最大深度
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def entropy(self, y):
        """
        計算entropy
        :param y: 目標變量
        :return: entropy
        """
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)  # 計算每個類別的機率
        # 利用講義公式計算entropy
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def best_split(self, X, y):
        """
        找到最佳分割點
        :param X: 特徵變量
        :param y: 目標變量
        :return: 最佳分割特徵的索引和閾值
        """
        m, n = X.shape
        if m <= 1:
            return None, None
        print("importance x(m,n)", X.shape)
        num_attributes = n
        min_entropy = float('inf')
        best_attribute, best_threshold = None, None

        for attribute in range(num_attributes):
            thresholds = np.unique(X[:, attribute])
            for threshold in thresholds: # 對每個閾值計算entropy，找出擁有最小entropy的閾值
                left_indices = X[:, attribute] <= threshold
                right_indices = ~left_indices

                left_entropy = self.entropy(y[left_indices])
                right_entropy = self.entropy(y[right_indices])

                entropy = (len(y[left_indices]) * left_entropy + len(y[right_indices]) * right_entropy) / len(y)

                # importance function 
                if entropy < min_entropy:
                    min_entropy = entropy
                    best_attribute = attribute
                    best_threshold = threshold
                    
        return best_attribute, best_threshold

    def Decision_Tree_Learning(self, X, y, depth=0):
        """
        建立決策樹
        :param X: 特徵變量
        :param y: 目標變量
        :param depth: 樹的深度
        :return: 樹的根節點
        """
        if self.max_depth is not None and depth == self.max_depth or len(set(y)) == 1:
            # 改變樹高 -> 如果樹的深度等於max_depth，則停止分割
            print("interrupt 達到最大深度")
            return Node(value=max(set(y), key=list(y).count))
        if self.min_samples_split is not None and len(y) < self.min_samples_split:
            # 剪枝 -> 如果樣本數小於min_samples_split，則停止分割
            print("interrupt 達到最小樣本數")
            return Node(value=max(set(y), key=list(y).count))
        
        attribute, threshold = self.best_split(X, y)
        print("best attribute", attribute)
        print("best threshold", threshold)
        
        if attribute is None:
            print("interrupt 找不到最佳分割點")
            return Node(value=max(set(y), key=list(y).count))

        # 判斷左右子樹
        left_indices = X[:, attribute] <= threshold 
        print("length of left 資料量", len(X[left_indices]))
        right_indices = ~left_indices
        print("length of right 資料量", len(X[right_indices]))
        print("進入左子樹")
        left_subtree = self.Decision_Tree_Learning( X[left_indices], y[left_indices], depth + 1)
        print("進入右子樹")
        right_subtree = self.Decision_Tree_Learning(X[right_indices], y[right_indices], depth + 1)

        return Node(attribute=attribute, threshold=threshold, left=left_subtree, right=right_subtree)

    def fit(self, X, y):
        """
        擬合模型
        :param X: 特徵變量
        :param y: 目標變量
        """
        self.tree = self.Decision_Tree_Learning(X, y)

    def predict_sample(self, tree, sample):
        """
        預測單個樣本
        :param tree: 決策樹
        :param sample: 單個樣本
        :return: 預測結果
        """
        if tree.value is not None:
            return tree.value

        if sample[tree.attribute] <= tree.threshold:
            return self.predict_sample(tree.left, sample)
        else:
            return self.predict_sample(tree.right, sample)

    def predict(self, X):
        """
        預測多個樣本
        :param X: 特徵變量
        :return: 預測結果
        """
        return np.array([self.predict_sample(self.tree, sample) for sample in X])

    # def importance(self, X, y):
    #     """
    #     計算特徵重要性
    #     :param X: 特徵變量
    #     :param y: 目標變量
    #     :return: 特徵重要性
    #     """
    #     attribute_importance = np.zeros(X.shape[1])
    #     total_samples = len(y)

    #     for attribute in range(X.shape[1]):
    #         permuted_X = X.copy()
    #         np.random.shuffle(permuted_X[:, attribute])
    #         y_pred = self.predict(permuted_X)
    #         accuracy = np.sum(y_pred == y) / total_samples
    #         attribute_importance[attribute] = accuracy

    #     return attribute_importance

    # def plurality_value(self, y):
    #     """
    #     找出最多數量的類別
    #     :param y: 目標變量
    #     :return: 最多數量的類別
    #     """
    #     unique_classes, counts = np.unique(y, return_counts=True)  # 找出所有類別和數量
    #     max_count = np.max(counts)  # 找出最多數量的類別
    #     plurality_values = unique_classes[counts == max_count]
    #     return plurality_values

    def accuracy(self, y_true, y_pred):
        """
        計算準確率
        :param y_true: 真實值
        :param y_pred: 預測值
        :return: 準確率
        """
        return np.sum(y_true == y_pred) / len(y_true)


In [19]:
# 加入訓練資料
model = DecisionTree(max_depth=5, min_samples_split=5)
model.fit(train_data.values, train_labels.values)
y_pred = model.predict(test_data.values)
print('Test Accuracy:', model.accuracy(test_labels.values, y_pred))

importance x(m,n) (460, 11)
attribute 9
threshold 97.0
length of left 資料量 206
length of right 資料量 254
進入左子樹
importance x(m,n) (206, 11)
attribute 1
threshold 0.19
length of left 資料量 75
length of right 資料量 131
進入左子樹
importance x(m,n) (75, 11)
attribute 9
threshold 57.0
length of left 資料量 61
length of right 資料量 14
進入左子樹
importance x(m,n) (61, 11)
attribute 0
threshold 0.0
length of left 資料量 40
length of right 資料量 21
進入左子樹
interrupt 達到最大深度
進入右子樹
importance x(m,n) (21, 11)
attribute 2
threshold 1.0
length of left 資料量 16
length of right 資料量 5
進入左子樹
interrupt 達到最大深度
進入右子樹
interrupt 達到最大深度
進入右子樹
importance x(m,n) (14, 11)
attribute 5
threshold 10.0
length of left 資料量 10
length of right 資料量 4
進入左子樹
importance x(m,n) (10, 11)
attribute 1
threshold 0.0
length of left 資料量 8
length of right 資料量 2
進入左子樹
interrupt 達到最大深度
進入右子樹
interrupt 達到最大深度
進入右子樹
interrupt 達到最大深度
進入右子樹
interrupt 達到最大深度
進入右子樹
importance x(m,n) (254, 11)
attribute 1
threshold 0.24
length of left 資料量 217
length of right 資料量 37
進入左子樹

In [ ]:
def _print_tree(tree):
    """
    印出決策樹
    :param model.tree: 決策樹
    """
    def print_node(tree, depth=0):
        if tree.value is not None:
            print("END depth", depth, tree.value)
        else:
            print("depth", depth, "attribute" ,tree.attribute,"threshold" ,tree.threshold)
            print_node(tree.left, depth + 1)
            print_node(tree.right, depth + 1)

    print_node(tree)
_print_tree(model.tree)


depth 0 attribute 9 threshold 97.0
depth 1 attribute 1 threshold 0.19
depth 2 attribute 9 threshold 57.0
depth 3 attribute 0 threshold 0.0
END depth 4 1
depth 4 attribute 2 threshold 1.0
depth 5 attribute 8 threshold 3.0
END depth 6 1
END depth 6 1
depth 5 attribute 2 threshold 2.0
END depth 6 0
END depth 6 1
depth 3 attribute 5 threshold 10.0
depth 4 attribute 1 threshold 0.0
depth 5 attribute 9 threshold 66.0
END depth 6 1
END depth 6 0
END depth 5 0
END depth 4 0
END depth 2 1
depth 1 attribute 1 threshold 0.24
depth 2 attribute 0 threshold 0.0
depth 3 attribute 5 threshold 0.0
END depth 4 1
END depth 4 0
depth 3 attribute 9 threshold 505.0
depth 4 attribute 10 threshold 689.0
depth 5 attribute 7 threshold 0.0
END depth 6 0
END depth 6 0
depth 5 attribute 8 threshold 26.0
END depth 6 1
END depth 6 0
depth 4 attribute 8 threshold 4.0
END depth 5 0
END depth 5 0
depth 2 attribute 5 threshold 9.0
depth 3 attribute 8 threshold 7.0
END depth 4 1
depth 4 attribute 1 threshold 0.31
END dep